In [1]:
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import path
import xmltodict
from myfunctions import event_separation , image_crop, image_crop_negative, poi
import tifffile

## Input data from a folder ##

In [2]:
files_dir = r'C:\Users\roumba\Documents\Software\deep-events\original_data'
images_dir = '221123_mtStayGold_U2OS_ZEISS_fl'
joined_path = os.path.join(files_dir, images_dir)
size=(2048,2048)
img,input_name,output_name,datacsv,pixel_size={},{},{},{},{}

for input_file in os.listdir(joined_path):
        joined_file_path = os.path.join(files_dir, images_dir, input_file)
        if '.tif' in input_file:
            tif  = tifffile.TiffFile(joined_file_path)
            mdInfoDict = xmltodict.parse(tif.ome_metadata, force_list={'Plane'})
            date, cell_type, dye, bf_fl, index  = input_file.split('_')
            number, ome, tiff= index.split('.')
            if '@PhysicalSizeX' in mdInfoDict['OME']['Image']['Pixels']:
                pixel_size_value=1
                pixel_size[int(number)-1]=mdInfoDict['OME']['Image']['Pixels']['@PhysicalSizeX']
            else:
                pixel_size_value=0
                pixel_size[int(number)-1]=1
            img[int(number)-1] = Image.open(joined_file_path)
            input_name[int(number)-1]= f'points_{date}_{cell_type}_{bf_fl}_{number}'
            output_name[int(number)-1]= f'image_{date}_{cell_type}_{bf_fl}_{number}'
            print('Loaded image:'+input_file)
        else:
            labels, index, bf_fl  = input_file.split('_')
            datacsv[int(index)-1] = pd.read_csv(joined_file_path)
            print('Loaded csv:'+ input_file)
print('Pixel scaling:',pixel_size)
max_number=int(len(datacsv))   

Loaded image:221123_U2OS_mitostaygold_GFP_1.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_10.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_11.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_12.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_2.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_3.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_4.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_5.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_6.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_7.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_8.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_9.ome.tif
Loaded csv:labels_10_fl.csv
Loaded csv:labels_11_fl.csv
Loaded csv:labels_12_fl.csv
Loaded csv:labels_1_fl.csv
Loaded csv:labels_2_fl.csv
Loaded csv:labels_3_fl.csv
Loaded csv:labels_4_fl.csv
Loaded csv:labels_5_fl.csv
Loaded csv:labels_6_fl.csv
Loaded csv:labels_7_fl.csv
Loaded csv:labels_8_fl.csv
Loaded csv:labels_9_fl.csv
Pixel scaling: {0: 1, 9: '0.103', 10: '0.103',

## Get all Gaussian Points of Interest ##

In [3]:
for x in range(max_number):
    csv=datacsv[x]
    im=img[x]
    for i in range(3,7,1):
        sigma=(i,i)
        s=sigma[0]
        in_name=f'{input_name[x]}_sigma{s}.tiff'
        framenum=im.n_frames
        poi(csv,in_name,sigma,size,framenum)

100%|██████████| 47/47 [00:12<00:00,  3.71it/s]


## Cut Images and Gaussian POIs ##

In [3]:
base_dir= r'C:\Users\roumba\Documents\Software\deep-events'
training_dir= r'C:\Users\roumba\Documents\Software\deep-events\training_data'
folder_name=images_dir+'_pos'
filepath= os.path.join(training_dir , folder_name)

for x in range(max_number):
    csv=datacsv[x]
    im=img[x]
    print('file number:',x)
    scale_value=float(pixel_size[x])
    csv['axis-1']=csv['axis-1']*(1/scale_value)
    csv['axis-2']=csv['axis-2']*(1/scale_value)
    for i in range(3,4,1):
        sigma=(i,i)
        s=sigma[0]
        print('sigma:',s)
        in_name=f'{input_name[x]}_sigma{s}.tiff'
        out_name=f'{output_name[x]}'
        out_name_g=f'{output_name[x]}_sigma{s}'
    
        list1=event_separation(csv)
        l=len(list1)
        image_crop(l,list1, csv, im,0, out_name,filepath)

        gauss=base_dir+f'\{in_name}'
        gauss_image=Image.open(gauss)
        image_crop(l,list1, csv, gauss_image,1, out_name_g,filepath)

file number: 0
sigma: 3
file number: 1
sigma: 3
file number: 2
sigma: 3
file number: 3
sigma: 3
file number: 4
sigma: 3
file number: 5
sigma: 3
file number: 6
sigma: 3
file number: 7
sigma: 3
file number: 8
sigma: 3
file number: 9
sigma: 3
file number: 10
sigma: 3
file number: 11
sigma: 3


## Cut Negative Frames ##

In [3]:
base_dir= r'C:\Users\roumba\Documents\Software\deep-events'
training_dir= r'C:\Users\roumba\Documents\Software\deep-events\training_data'
folder_name=images_dir+'_neg'
filepath= os.path.join(training_dir , folder_name)
from myfunctions import image_crop_negative

for x in range(max_number):
    csv=datacsv[x]
    im=img[x]
    print('file number:',x)
    out_name=f'{output_name[x]}'

    list1=event_separation(csv)
    l=len(list1)
    image_crop_negative(l,list1, csv, im, out_name,filepath)

file number: 0
file number: 1
file number: 2
file number: 3
file number: 4
file number: 5
file number: 6
file number: 7
file number: 8
file number: 9
file number: 10
file number: 11
